In [1]:
pip install requests beautifulsoup4 selenium openpyxl pandas

   ---------------------------------------- 0.0/147.9 kB ? eta -:--:--
   -------- ------------------------------- 30.7/147.9 kB ? eta -:--:--
   ---------------------------------------- 147.9/147.9 kB 2.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/9.4 MB ? eta -:--:--
   - -------------------------------------- 0.2/9.4 MB 5.0 MB/s eta 0:00:02
   --- ------------------------------------ 0.9/9.4 MB 9.3 MB/s eta 0:00:01
   -------- ------------------------------- 2.1/9.4 MB 16.4 MB/s eta 0:00:01
   ----------------- ---------------------- 4.2/9.4 MB 24.2 MB/s eta 0:00:01
   ------------------------- -------------- 6.0/9.4 MB 29.5 MB/s eta 0:00:01
   ----------------------------- ---------- 6.9/9.4 MB 27.6 MB/s eta 0:00:01
   ------------------------------ --------- 7.2/9.4 MB 25.7 MB/s eta 0:00:01
   -------------------------------------- - 9.1/9.4 MB 25.3 MB/s eta 0:00:01
   ---------------------------------------- 9.4/9.4 MB 23.2 MB/s eta 0:00:00
   --------------


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
import os

In [10]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

# Selenium 설정
options = webdriver.ChromeOptions()
options.add_argument('user-agent=Your User-Agent String')
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
driver.get("https://link.springer.com/journal/13428/volumes-and-issues")

# BeautifulSoup 설정
soup = BeautifulSoup(driver.page_source, 'html.parser')

# 데이터 저장용 리스트
data = []

# 2022년부터 2024년까지의 정보를 가져오기 위한 반복문
for year in range(2022, 2025):
    issues = soup.find_all('a', href=True, string=str(year))
    for issue in issues:
        issue_link = "https://link.springer.com" + issue['href']
        driver.get(issue_link)
        
        # 페이지 로딩 대기
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'issue-item')))
        
        issue_soup = BeautifulSoup(driver.page_source, 'html.parser')
        
        articles = issue_soup.find_all('article')
        issue_number = issue_soup.find('h1', class_='u-no-margin').text.strip()
        
        for article in articles:
            title = article.find('h2').text.strip()
            authors = [author.text for author in article.find_all('span', class_='authors__name')]
            data.append([year, issue_number, title, ', '.join(authors)])

# 데이터를 DataFrame으로 변환
df = pd.DataFrame(data, columns=['Year', 'Issue', 'Title', 'Authors'])


path = 'D:/journal/'
# 엑셀 파일로 저장
df.to_excel(os.path.join(path,'springer_journal_data.xlsx'), index=False)

# 드라이버 종료
driver.quit()